## NetCDF basics in Python

1. Read data from a NetCDF file
2. Create a simple contour plot

In [ ]:
# core scientific libraries
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# secondary libraries
import netCDF4 as nc


sys.path.append('../modules')
from plotter import simple_contour_plot
from function_example import date_range

In [ ]:
## test date_range function

test_list_dates = date_range('1997-01-01', '1997-01-31', '%Y-%M-%d', 1, 'days')
test_list_dates

### Reading data from a NetCDF file

In [ ]:
# Set filepath
datdir = '../sample-data/'
filename = 'erai.mon.t2m.2000-2005.nc'
filepath = datdir + filename
print(filepath, '\n')

# Open NetCDF Dataset object
f = nc.Dataset(filepath,'r')

# List variable names
print("Variables:")
print(f.variables.keys(),'\n')

# Show dimension names and sizes
print("Dimensions:")
print(f.dimensions.items(),'\n')

# Print all file metadata
#print(f)   # or run 'ncdump -h <filename>' in terminal

In [ ]:
# Read the variables
lons = f.variables['longitude'][:] # [:] syntax stores data in np arrays
lats = f.variables['latitude'][:]
t2m  = f.variables['t2m'][:]
time = f.variables['time']

# Inspect your data
print(lats.shape, lons.shape) # print array sizes
print(t2m.shape)  
print(lats[0:10])  # print some values

# Retrieve attributes
t2m_units    = f.variables['t2m'].units
t2m_longname = f.variables['t2m'].long_name
t2m_missing  = f.variables['t2m'].missing_value
print(t2m_longname)

### Time coordinate handling

**`num2date(`**`times, units, calendar='standard')`: <br>
converts numeric time values to datetime objects, given the specified `units` and `calendar`. Units are generally written in the format `"hours since YY:MM:DD hh-mm-ss"` or "\{time unit\} since \{some reference time\}"

**`date2num(`**`dates, units, calendar='standard')`: <br>
converts datetime objects to numeric values of time (the reverse of num2date)


In [ ]:
# convert times to a datetime object
time_units = f.variables['time'].units
dates = nc.num2date(time[:], time_units)

### Simple arithmetic operations

In [ ]:
# Long-term time average
t2m_mean = np.mean(t2m, axis=0)
print(t2m_mean.shape)


### Contour plot
Using Cartopy with Matplotlib

Plotting resources:
* [Cartopy projection list](https://scitools.org.uk/cartopy/docs/latest/crs/projections.html#cartopy-projections)
* [Matplotlib colormaps](https://matplotlib.org/tutorials/colors/colormaps.html)

In [ ]:
# mapcrs - what want in
# datacrs - what comes in

In [ ]:
simple_contour_plot(t2m_mean, lons, lats, sequential = True)

In [ ]:
# Data for plotting
data = t2m_mean

# Set map projection
proj = ccrs.PlateCarree()

# Set up figure and axes
fig = plt.figure(figsize=(7, 5))
ax = fig.add_subplot(1, 1, 1, projection=proj)

# Add data
p = ax.contourf(lons, lats, data, transform=proj,
                cmap='YlOrRd', extend='both')

# Add plot elements
ax.coastlines()
ax.gridlines()
plt.title('Average 2m Temperature (2000-2005)')

# Add colorbar
cbar = plt.colorbar(p, orientation='horizontal',
                    shrink=0.85, pad=0.05, 
                    label=t2m_units)

# Save to file
plt.savefig('plotfile.png')

# Show plot
plt.show()